In [ ]:
import os, sys
import pandas as pd
import numpy as np
import re
import resource
import cplex

from cobra.io import read_sbml_model, write_sbml_model
from cobra.util.solver import solvers
from cobra.core.configuration import Configuration
from cobra.flux_analysis import find_blocked_reactions

from urllib.request import urlretrieve
from cobamp.utilities.parallel import batch_run, cpu_count
from troppo.methods_wrappers import ReconstructionWrapper
from troppo.omics.core import OmicsMeasurementSet, OmicsContainer
from troppo.methods_wrappers import integration_strategy_map
from troppo.omics.integration import MINSUM, MINMAX
from itertools import product

from cobra.flux_analysis import fastcc

In [ ]:
 Configuration.solver = solvers['cplex']

In [ ]:
arg = [argval.split('=')[1] for argval in sys.argv[1:] if '-config=' in argval]
mp = '-no-mp' not in sys.argv[1:]

In [ ]:
# Load Recon3D model
model = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')
model_cobra = read_sbml_model('/home/jiang817/cybergut_project/General_models/Recon3D.xml')

In [ ]:
# Remove blocked reactions in the Recon3D
blocked = find_blocked_reactions(model)
model.remove_reactions(blocked)


In [ ]:
# Set exchagne reaction boundries as (-100, 100)
for i in range(len(model.reactions)):
    if model.reactions[i].id.startswith('EX_'):
        model.reactions[i].bounds = (-100, 100)


In [ ]:
# Load gene expression data 
exp_data = pd.read_csv('/home/jiang817/cybergut_project/Draft_model/Recon3D_input/Recon3D_XML_tINIT_INPUT/expressiondata_tinit.csv')

In [ ]:
# Convert epxression data into dictionary as input
data_dict = {}
exp_data.iloc[1,0]
for i in range(len(exp_data)):
    data_dict[exp_data.iloc[i, 0]] = exp_data.iloc[i, 1]
data_dict

In [ ]:
# Set protected reactions
protected = ['BIOMASS_maintenance', 'ATPS4mi']

In [ ]:
# Parameter setup
params = {
            'algorithms':['tinit','fastcore'],
            'strategies': {
                'tinit': integration_strategy_map['adjusted_score'](protected),
                'fastcore': integration_strategy_map['default_core'](0, protected)
            },
            'functions':{'minmax': MINMAX, 'minsum': MINSUM},
    }

 def reconstruct_model(options, params):
    print('\tResource statistics before reconstruction with',options[0],':', resource.getrusage(resource.RUSAGE_SELF))
    alg, d, func = options
    data_dict, aofunc = d, params['functions'][func]

    oc_sample = OmicsContainer(omicstype='transcriptomics', condition='x',
                                data=data_dict, nomenclature='custom')

    return params['rw'].run_from_omics(omics_data=oc_sample, algorithm=alg, and_or_funcs=aofunc,
                        integration_strategy=params['strategies'][alg], solver='GUROBI', raise_errors=False)

rw = ReconstructionWrapper(model, ttg_ratio=9999)
params['rw'] = rw

algo, intfx  = ('tinit', 'minmax')

In [ ]:
# Run tINIT
reconstructions = reconstruct_model((algo, data_dict, intfx), params)

In [ ]:
# Collect removed reactions
num_remove = 0
for i in reconstructions.values():
    if i == False:
        num_remove += 1
print(num_remove)

rm_rxn = []
for (k, v) in reconstructions.items():
    if v == False:
        print(k)
        rm_rxn.append(k)

In [ ]:
model_tINIT  = model_cobra

In [ ]:
model_tINIT.remove_reactions(rm_rxn, True)

In [ ]:
write_sbml_model(model_tINIT, "/home/jiang817/cybergut_project/Draft_model/Draft_model_with_tINIT/draft_reconstruction_tINIT_BIGG.xml")

In [ ]:
# run fastcc to obtain consistent reconstruction 
model_fastcc = fastcc(model_tINIT)

# save consistent reconstuction
write_sbml_model(model_fastcc, '/home/jiang817/cybergut_project/Gapfilling/fastcc_outputs/reconstruction_tINIT_fastcc.xml')